In [ ]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
# model = YOLO(r"C:\Users\wajah\Downloads\ALL\runs\detect\train5\weights\last.pt")

from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = YOLO("yolo11m.pt")

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data=r"D:\ALL\customdatabase.yaml", epochs=100, imgsz=640,batch=-1)

# results = model.train(resume=True)

# Run inference with the YOLO11n model on the 'bus.jpg' image
# results = model("path/to/bus.jpg")

In [ ]:
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn,fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import faster_rcnn
# from torchvision.models.detection import BackboneWithFPN
import numpy as np
from torchvision.datasets import CocoDetection
import matplotlib.pyplot as plt
from torchvision import models, datasets, tv_tensors
from helpers import plot
from torchvision.transforms import v2
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
from torch.utils.data import ConcatDataset, DataLoader

# Ensure you have the correct COCO dataset, or use a different dataset
# This example assumes you're using COCO format dataset, but similar for VOC
coco_gt = COCO(r"C:\Users\wajah\Downloads\ALL\H_100X_C2\json_labels\test.json")

# Data Transformations
transforms = v2.Compose(
    [
        # v2.RandomApply(transforms=[v2.GaussianBlur(kernel_size=(5, 5), sigma=(0.5, 3.0))],p=0.5),
        v2.ToImage(),
        # v2.RandomPhotometricDistort(p=1),
        # v2.RandomRotation(degrees=(-30, 30)),
        # v2.RandomErasing(),
        # v2.JPEG((10)),
        # v2.RandomApply(transforms=[v2.JPEG((10))],p=1.0),
        # v2.RandomApply(transforms=[v2.JPEG((20,60))],p=0.3),
        v2.RandomApply(transforms=[v2.RandomIoUCrop()],p=0.5),
        v2.RandomPerspective(distortion_scale=0.3,fill={tv_tensors.Image: (123, 117, 104), "others": 0}, p=0.5),
        v2.RandomZoomOut(fill={tv_tensors.Image: (123, 117, 104), "others": 0},side_range=(1.0, 2.0),p=0.5),
        v2.RandomApply(transforms=[v2.ColorJitter(brightness=0.5,contrast=0.3,saturation=0.3)],p=0.5),
        v2.RandomHorizontalFlip(p=1),
        v2.Resize((640,640),antialias=True),
        v2.SanitizeBoundingBoxes(),
        v2.ToDtype(torch.float32, scale=True),
        # v2.RandomApply(transforms=[v2.GaussianNoise(sigma=0.05)],p=0.5),
    ]
)

test_transforms = v2.Compose(
    [
        v2.ToImage(),
        # v2.SanitizeBoundingBoxes(),
        v2.ToDtype(torch.float32, scale=True),
    ]
)


# Load COCO Dataset (you can change it for VOC dataset)
train_dataset = CocoDetection(
    root=r"D:\ALL\images\train",
    annFile=r"C:\Users\wajah\Downloads\ALL\H_100X_C2\json_labels\train.json",
    transforms=transforms
)

# train_dataset_av = CocoDetection(
#     root=r'C:\Users\wajah\Downloads\ALL\H_100X_C2\adv_images\train',
#     annFile=r"C:\Users\wajah\Downloads\ALL\H_100X_C2\json_labels\train.json",
#     transforms=transforms
# )


dataset_0 = datasets.wrap_dataset_for_transforms_v2(train_dataset, target_keys=("boxes", "labels"))
# dataset_1 = datasets.wrap_dataset_for_transforms_v2(train_dataset_av, target_keys=("boxes", "labels"))
# print(len(dataset_0))
# combined_dataset = ConcatDataset([dataset_0, dataset_1])
# print(len(combined_dataset))
# Load COCO Dataset (you can change it for VOC dataset)
test_dataset = CocoDetection(
    root=r"C:\Users\wajah\Downloads\ALL\H_100X_C2\images\test",
    annFile=r"C:\Users\wajah\Downloads\ALL\H_100X_C2\json_labels\test.json",
    transforms=test_transforms
)

test_dataset = datasets.wrap_dataset_for_transforms_v2(test_dataset, target_keys=("boxes", "labels","image_id"))



train_loader = torch.utils.data.DataLoader(
    dataset_0,
    batch_size=4,
    shuffle=True,
    # We need a custom collation function here, since the object detection
    # models expect a sequence of images and target dictionaries. The default
    # collation function tries to torch.stack() the individual elements,
    # which fails in general for object detection, because the number of bounding
    # boxes varies between the images of the same batch.
    collate_fn=lambda batch: tuple(zip(*batch)),
)


test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    # We need a custom collation function here, since the object detection
    # models expect a sequence of images and target dictionaries. The default
    # collation function tries to torch.stack() the individual elements,
    # which fails in general for object detection, because the number of bounding
    # boxes varies between the images of the same batch.
    collate_fn=lambda batch: tuple(zip(*batch)),
)


In [ ]:
sample = test_dataset[0]
img, target = sample
print(f"{type(img) = }\n{type(target) = }\n{target.keys() = }")
print(f"{type(target['boxes']) = }\n{type(target['labels']) = }")
plot([dataset_0[1], dataset_0[2]])
print(len(test_loader))

In [ ]:
def create_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='COCO_V1')
    # model= torchvision.models.detection.retinanet_resnet50_fpn_v2(weights='COCO_V1')
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    
    return model

num_classes = 15
model = create_model(num_classes)

# model.load_state_dict(torch.load(r"C:\Users\wajah\Downloads\ALL\best_model(FasterRCNN)_adamw_640_50.pth"))
# from torchvision.models import resnet50, ResNet50_Weights

# model= torchvision.models.detection.retinanet_resnet50_fpn_v2(weights=None,num_classes=15, weights_backbone = ResNet50_Weights.IMAGENET1K_V2)

In [ ]:
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
# from torchvision.models import resnet50, ResNet50_Weights

# # Setup
# backbone = resnet_fpn_backbone(backbone_name='resnet50', weights="IMAGENET1K_V1")
# model = FasterRCNN(backbone=backbone, num_classes=15,min_size=640,max_size=640)


# import torchvision
# from torchvision.models.detection import RetinaNet
# from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
# from torchvision.models import resnet50, ResNet50_Weights

# # Load a pretrained ResNet101 backbone with FPN
# backbone = resnet_fpn_backbone(
#     backbone_name='resnet50',
#     weights=ResNet50_Weights.IMAGENET1K_V1,
#     # trainable_layers=3  # Optional: freeze earlier layers
# )

# # Create the RetinaNet model
# model = RetinaNet(backbone=backbone, num_classes=15,min_size=640,max_size=640)
# print(model)

In [ ]:
import torch
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

from tqdm import tqdm

def train_one_epoch(model, train_loader, optimizer, scaler, accumulation_steps, device):
    model.train()
    train_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training", unit="batch")

    optimizer.zero_grad()

    for i, (images, targets) in progress_bar:
        images = [img.to(device) for img in images]
        
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Normalize loss for accumulation
        loss = losses / accumulation_steps
        loss.backward()

        # Only update weights every `accumulation_steps` iterations
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            optimizer.step()
            optimizer.zero_grad()

        train_loss += losses.item()  # Add actual (un-normalized) loss
        progress_bar.set_postfix(loss=losses.item())

    avg_train_loss = train_loss / len(train_loader)
    return avg_train_loss


from pycocotools.cocoeval import COCOeval
import numpy as np

def validate(model, val_loader, device, coco_gt):
    model.eval()  # Use eval mode for evaluation
    predictions = []
    image_ids = []
    annotation_id = 0  # Unique ID across all predictions

    with torch.no_grad():
        for images, targets in val_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)

            for i, output in enumerate(outputs):
                image_id = targets[i]['image_id']
                boxes = output['boxes'].cpu().numpy()
                scores = output['scores'].cpu().numpy()
                labels = output['labels'].cpu().numpy()

                for box, score, label in zip(boxes, scores, labels):
                    x_min, y_min, x_max, y_max = box
                    width = x_max - x_min
                    height = y_max - y_min

                    predictions.append({
                        'image_id': image_id,
                        'category_id': int(label),
                        'bbox': [x_min, y_min, width, height],
                        'score': float(score)
                    })

    # Evaluate with COCO API
    coco_dt = coco_gt.loadRes(predictions)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    mAP_50_95 = coco_eval.stats[0]  # mAP@[0.50:0.95]
    mAP_50 = coco_eval.stats[1]     # mAP@0.50

    return mAP_50, mAP_50_95

def train_and_evaluate(model, train_loader, val_loader, optimizer, scaler, accumulation_steps,
                       lr_scheduler, num_epochs, device, coco_gt, model_save_path='best_model.pth'):
    best_map = 0.0
    epochs_no_improve = 0
    patience = 50  # Early stopping patience
    # avg_train_loss=0.0
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Training
        avg_train_loss = train_one_epoch(model, train_loader, optimizer, scaler, accumulation_steps, device)

        # Validation - returns mAP@50 and mAP@[.50:.95]
        mAP_50, mAP_50_95 = validate(model, val_loader, device, coco_gt)

        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f} - "
              f"mAP@50: {mAP_50:.4f} - mAP@[.50:.95]: {mAP_50_95:.4f}")

        # Save the model if mAP@[.50:.95] improved
        if mAP_50 > best_map:
            print("Validation mAP improved, saving the model...")
            best_map = mAP_50
            torch.save(model.state_dict(), model_save_path)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Early stopping
        if epochs_no_improve >= patience:
            print(f"Early stopping after {epochs_no_improve} epochs without improvement.")
            break

        # Update learning rate scheduler
        lr_scheduler.step()

    print("Training complete!")




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def find_lr(model, train_loader, optimizer, device, init_value=1e-7, final_value=10., beta=0.98):
    num = len(train_loader) - 1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    avg_loss = 0.
    best_loss = float('inf')
    batch_num = 0
    losses = []
    log_lrs = []

    model.train()

    for batch_num, (images, targets) in enumerate(train_loader):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        avg_loss = beta * avg_loss + (1 - beta) * loss.item()
        smoothed_loss = avg_loss / (1 - beta**(batch_num + 1))

        log_lrs.append(np.log10(lr))
        losses.append(smoothed_loss)

        loss.backward()
        optimizer.step()

        lr *= mult
        optimizer.param_groups[0]['lr'] = lr

        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            break

        if smoothed_loss < best_loss or batch_num == 0:
            best_loss = smoothed_loss

    return log_lrs, losses


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer and scheduler
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.AdamW(params, lr=1e-3, weight_decay=1e-4)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Mixed precision scaler
scaler = GradScaler()

# Gradient accumulation setup
accumulation_steps = 4
num_epochs = 100
# Define the save path for the best model
model_save_path = 'Faster_v2_model_adamw_640_50.pth'

# log_lrs, losses= find_lr(model, train_loader, optimizer, device, init_value=1e-5, final_value=10.0, beta=0.98)

# # # log_lrs, losses = find_lr(model, train_loader, optimizer, device)
# # # 
# plt.plot(log_lrs, losses)
# plt.xlabel("Log10 Learning Rate")
# plt.ylabel("Loss")
# plt.title("Learning Rate Finder")
# plt.grid(True)
# plt.show()

# Train and evaluate
train_and_evaluate(model, train_loader, test_loader, optimizer, scaler, accumulation_steps, lr_scheduler, num_epochs, device,coco_gt, model_save_path)


In [ ]:
C
train_and_evaluate(model, train_loader, test_loader, optimizer, scaler, accumulation_steps, lr_scheduler, num_epochs, device,coco_gt, model_save_path)

In [ ]:
torch.save(model.state_dict(), "final_model.pth")

In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.datasets import CocoDetection
from torchvision.transforms import v2 as T
from torchvision import datasets
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import torchvision

# ----------------------------
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Define test transforms
test_transforms = T.Compose([
    T.ToImage(),
    T.ToDtype(torch.float32, scale=True),
    # T.Resize((512, 512)),
    # T.ToTensor()
])

# ----------------------------
# Load COCO annotation and dataset
coco = COCO(r"C:\Users\wajah\Downloads\ALL\H_40X_C2\json_labels\test.json")

test_dataset = CocoDetection(
    root=r'C:\Users\wajah\Downloads\ALL\H_40X_C2\images\test',
    annFile=r"C:\Users\wajah\Downloads\ALL\H_40X_C2\json_labels\test.json",
    transforms=test_transforms
)

# Optional: wrap dataset for TorchVision v2
dataset = datasets.wrap_dataset_for_transforms_v2(test_dataset, target_keys=("boxes", "labels", "image_id"))

# ----------------------------
# Load Faster R-CNN model
model =torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=15)  # <-- change num_classes based on your dataset

# fasterrcnn_resnet50_fpn
# model.load_state_dict(torch.load("final_model.pth", map_location=device))
model.load_state_dict(torch.load(r"C:\Users\wajah\Downloads\ALL\Faster_v2_model_adamw_640_50.pth", map_location=device))
# Define the save path for the best model
# model_save_path = 'best_model_adamw_640_50.pth'

model.to(device)
model.eval()

# ----------------------------
# Run inference and build predictions in COCO format
predictions = []

for idx in range(len(dataset)):
    image, target = dataset[idx]

    image = image.to(device)
    image_id = target["image_id"].item() if isinstance(target["image_id"], torch.Tensor) else target["image_id"]

    with torch.no_grad():
        outputs = model([image])

    output = outputs[0]

    for box, score, label in zip(output["boxes"], output["scores"], output["labels"]):
        x1, y1, x2, y2 = box.tolist()
        width = x2 - x1
        height = y2 - y1
        coco_box = [x1, y1, width, height]

        predictions.append({
            "image_id": image_id,
            "category_id": int(label.item()),
            "bbox": coco_box,
            "score": float(score.item())
        })

# ----------------------------
# Load predictions into COCOeval
coco_dt = coco.loadRes(predictions)
coco_eval = COCOeval(coco, coco_dt, iouType='bbox')

# Evaluate and summarize
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()


In [ ]:
torch.save(model, 'model.pth')

In [ ]:
import json
import os
from PIL import Image

# --- CONFIG ---
input_dir = r'C:\Users\wajah\Downloads\ALL\H_100X_C2\images\train'         # Folder with images
label_dir = r"C:\Users\wajah\Downloads\ALL\H_100X_C2\txt_labels\WBC_Detection\train"         # Folder with YOLO .txt files
output_dir = r'C:\Users\wajah\Downloads\ALL\H_100X_C2'   # Folder to save COCO JSON

# --- CREATE 14 CATEGORY ENTRIES ---
categories = [{"id": i + 1, "name": f"class_{i}", "supercategory": "none"} for i in range(14)]

# --- INIT COCO STRUCTURE ---
coco_dataset = {
    "info": {"description": "YOLO to COCO Converted Dataset"},
    "licenses": [],
    "categories": categories,
    "images": [],
    "annotations": []
}

annotation_id = 0
image_id = 0

# --- LOOP OVER IMAGES ---
for image_file in os.listdir(input_dir):
    if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue  # skip non-images

    image_path = os.path.join(input_dir, image_file)
    label_path = os.path.join(label_dir, os.path.splitext(image_file)[0] + ".txt")

    if not os.path.exists(label_path):
        continue  # skip if label not found

    # Read image to get dimensions
    with Image.open(image_path) as img:
        width, height = img.size

    image_id += 1
    coco_dataset["images"].append({
        "id": image_id,
        "file_name": image_file,
        "width": width,
        "height": height
    })

    # Read YOLO labels
    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        class_id, x_center, y_center, w, h = map(float, parts)
        x_center *= width
        y_center *= height
        w *= width
        h *= height
        x_min = x_center - w / 2
        y_min = y_center - h / 2

        annotation_id += 1
        coco_dataset["annotations"].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": int(class_id) + 1,  # YOLO class 0 → COCO category 1
            "bbox": [x_min, y_min, w, h],
            "area": w * h,
            "iscrowd": 0
        })

# --- SAVE JSON ---
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, 'annotations.json'), 'w') as f:
    json.dump(coco_dataset, f, indent=4)

print(f"✅ COCO annotation file saved to: {os.path.join(output_dir, 'annotations.json')}")


In [ ]:

# from ultralytics import YOLO
# import torch.optim as optim
# import torch.nn as nn
# import torch.nn.functional as F
# import torch
# import torchvision.models as models

# model = YOLO("yolo11l-cls.pt")  # load a pretrained model (recommended for training)

# # Train the model
# results = model.train(data=r"D:\ALL", epochs=100, imgsz=224,degrees=30.0)

In [ ]:
# import os
# import cv2

# # Paths
# image_dir = r"C:\Users\wajah\Downloads\ALL\H_100X_C2\images\train"
# label_dir = r"C:\Users\wajah\Downloads\ALL\H_100X_C2\labels_mul\train"
# output_dir = r"D:\ALL\train1/"

# # Create output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# # Loop through all annotation files
# for label_file in os.listdir(label_dir):
#     if not label_file.endswith(".txt"):
#         continue

#     image_name = os.path.splitext(label_file)[0] + ".png"
#     image_path = os.path.join(image_dir, image_name)
#     label_path = os.path.join(label_dir, label_file)

#     if not os.path.exists(image_path):
#         print(f"Image {image_path} not found.")
#         continue

#     # Load image
#     img = cv2.imread(image_path)
#     h_img, w_img, _ = img.shape

#     # Read label file
#     with open(label_path, "r") as f:
#         lines = f.readlines()

#     for idx, line in enumerate(lines):
#         parts = line.strip().split()
#         if len(parts) < 5:
#             continue  # skip malformed lines

#         class_id = parts[0]
#         cx, cy, bw, bh = map(float, parts[1:5])

#         # Convert normalized to absolute coordinates
#         x_center = int(cx * w_img)
#         y_center = int(cy * h_img)
#         box_w = int(bw * w_img)
#         box_h = int(bh * h_img)

#         # Determine the size to make it square
#         side_length = max(box_w, box_h)

#         # Calculate new square box coordinates
#         x1 = x_center - side_length // 2
#         y1 = y_center - side_length // 2
#         x2 = x1 + side_length
#         y2 = y1 + side_lengthimport torch.optim as optim
# import torch.nn as nn
# import torch.nn.functional as F

# import torch
# import torchvision.models as models

# # Download and load the pre-trained ResNet-18 model
# net = models.resnet50(pretrained=True)

#         # Clamp to image boundaries
#         x1 = max(0, x1)
#         y1 = max(0, y1)
#         x2 = min(w_img, x2)
#         y2 = min(h_img, y2)

#         # Adjust again if clamping changes size
#         crop = img[y1:y2, x1:x2]
#         if crop.shape[0] != crop.shape[1]:
#             side = min(crop.shape[:2])
#             crop = crop[:side, :side]

#         if crop.size == 0:
#             print(f"Skipping empty crop: {image_name}, box {idx}")
#             continue

#         # Create class-specific output folder
#         class_folder = os.path.join(output_dir, f"class_{class_id}")
#         os.makedirs(class_folder, exist_ok=True)

#         # Save cropped and square image
#         out_path = os.path.join(class_folder, f"{os.path.splitext(image_name)[0]}_{idx}.jpg")
#         cv2.imwrite(out_path, crop)
#         print(f"Saved: {out_path}")


In [ ]:
# import os

# input_root = r"C:\Users\wajah\Downloads\ALL\H_100X_C2\labels"
# output_root = "labels_roi"

# for split in ["train", "test"]:
#     input_folder = os.path.join(input_root, split)
#     output_folder = os.path.join(output_root, split)
#     os.makedirs(output_folder, exist_ok=True)

#     for filename in os.listdir(input_folder):
#         if not filename.endswith(".txt"):
#             continue

#         input_path = os.path.join(input_folder, filename)
#         output_path = os.path.join(output_folder, filename)

#         with open(input_path, "r") as infile, open(output_path, "w") as outfile:
#             for line in infile:
#                 parts = line.strip().split()
#                 if len(parts) >= 5:
#                     # Convert class_id to 0, keep bbox and extra info
#                     new_line = "0 " + " ".join(parts[1:]) + "\n"
#                     outfile.write(new_line)

# print("✅ All annotations in 'train' and 'test' converted to single-class format for ROI detection.")


In [ ]:
# import os
# import csv

# root_dir = r"D:\ALL\test"  # Main folder containing class-named subfolders
# output_csv = r"D:\ALL\test_labels.csv"

# with open(output_csv, "w", newline="") as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(["filename", "label"])  # Header

#     for label in sorted(os.listdir(root_dir)):
#         class_path = os.path.join(root_dir, label)
#         if not os.path.isdir(class_path):
#             continue

#         for img_file in os.listdir(class_path):
#             if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
#                 img_path = os.path.join(label, img_file)
#                 writer.writerow([img_path, label.split("_")[1]])

# print(f"✅ CSV file saved: {output_csv}")


In [ ]:
!nvidia-smi

In [ ]:
# import torch.optim as optim
# import torch.nn as nn
# import torch.nn.functional as F

# import torch
# import torchvision.models as models

# # Download and load the pre-trained ResNet-18 model
# net = models.resnet50(pretrained=True)
# print(net)

In [ ]:
import json
import os

def convert_to_yolo(json_path, output_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)

    image_info = {img['id']: img for img in data['images']}
    os.makedirs(output_dir, exist_ok=True)

    # Create a mapping from original category_id to 0-based class index
    category_id_to_index = {cat['id']: idx for idx, cat in enumerate(sorted(data['categories'], key=lambda x: x['id']))}

    for ann in data['annotations']:
        image_id = ann['image_id']
        if isinstance(image_id, int):
            img = image_info[image_id]
            filename = img['file_name']
        else:
            filename = image_id
            img = next((i for i in image_info.values() if i['file_name'] == filename), None)
            if img is None:
                continue

        img_w, img_h = img['width'], img['height']
        x, y, w, h = ann['bbox']

        # Convert to YOLO format
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        # Map original category_id to index in 0–13
        class_id = category_id_to_index[ann['category_id']]

        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(output_dir, label_filename)

        with open(label_path, 'a') as f:
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

    print(f"✅ Converted {len(data['annotations'])} annotations to YOLO format in '{output_dir}'")
    print(f"✅ Class IDs used: {list(category_id_to_index.values())}")

# Example usage:
convert_to_yolo(r"D:\ALL\Blance_augmented.json", r"D:\ALL\labels\train")

In [ ]:
import json

def extract_classes(json_path, output_file="classes.txt"):
    with open(json_path, 'r') as f:
        data = json.load(f)

    if "categories" not in data:
        print("No 'categories' field found in JSON.")
        return

    categories = sorted(data['categories'], key=lambda x: x['id'])
    with open(output_file, 'w') as f:
        for category in categories:
            f.write(f"{category['name']}\n")

    print(f"✅ Saved {len(categories)} classes to '{output_file}'")

# Example usage:
extract_classes(r"D:\ALL\Blance_augmented.json")
